# Sensorless Drive Diagnosis

> In this example, the main focus is the classification of individual states of a motor.

In [ ]:
# |hide
from nbdev.showdoc import *

In [ ]:
# | hide
import torch
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter
import numpy as np
import pandas as pd
import copy

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from matplotlib import pyplot as plt
import seaborn as sns

from mlmvn.layers import FirstLayer, HiddenLayer, OutputLayer, cmplx_phase_activation
from mlmvn.loss import ComplexMSELoss, ComplexMSE_adjusted_error
from mlmvn.optim import MySGD, ECL
from pathlib import Path
from clearml import Task, Logger

In [ ]:
# | hide
# --- helper functions ---
def reverse_one_hot(x, neuronCats):
    a = np.zeros(len(x))
    x = torch.detach(x)
    for i in range(len(x)):
        a[i] = torch.max(x[i]) - 1 + np.argmax(x[i]) * neuronCats
    return a


def accuracy(out, yb):
    out = out.type(torch.double)
    yb = yb.type(torch.double)
    x = 0
    for i in range(len(out)):
        x += torch.equal(out[i], yb[i])
    return x / len(out)


def prepare_data(x_train, x_valid, y_train, y_valid, neuronCats):
    # one-hot encoding
    numSamples, numFeatures = x_valid.shape
    y_valid_int = y_valid
    y2 = y_valid + 1  # auxiliary variable so that classes start at 1 and not 0
    numClasses = max(y2)
    target_ids = range(numClasses)
    no = int(np.ceil(numClasses / neuronCats))  # number of output neurons
    if no != 1:
        y_valid = torch.zeros(numSamples, no)
        for i in range(numSamples):
            k = int(np.ceil(y2[i] / neuronCats)) - 1
            c = np.mod((y2[i] - 1), neuronCats) + 1
            y_valid[i, k] = c
    numSamples, numFeatures = x_train.shape
    y_train_int = y_train
    y2 = y_train + 1  # auxiliary variable so that classes start at 1 and not 0
    if no != 1:
        y_train = torch.zeros(numSamples, no)
        for i in range(numSamples):
            k = int(np.ceil(y2[i] / neuronCats)) - 1
            c = np.mod((y2[i] - 1), neuronCats) + 1
            y_train[i, k] = c
    del y2

    # Convert numpy arrays into torch tensors
    x_train, y_train, x_valid, y_valid = map(
        torch.tensor, (x_train, y_train, x_valid, y_valid)
    )
    if y_train.size().__len__() == 1:
        y_train = torch.unsqueeze(y_train, 1)
        y_valid = torch.unsqueeze(y_valid, 1)

    # convert angles to complex numbers on unit-circle
    x_train = torch.exp(1.0j * x_train)
    x_valid = torch.exp(1.0j * x_valid)

    return x_train, x_valid, y_train, y_valid


def get_splitted_data(X, y, neuronCats):
    x_train, x_valid, y_train, y_valid = train_test_split(
        X, y, train_size=46806, random_state=42
    )
    x_train, x_valid, y_train, y_valid = prepare_data(
        x_train, x_valid, y_train, y_valid, neuronCats
    )

    return x_train, x_valid, y_train, y_valid


def get_splitted_data_by_index(X, y, neuronCats, train_index, test_index):
    x_train, x_valid = X[train_index], X[test_index]
    y_train, y_valid = y[train_index], y[test_index]
    x_train, x_valid, y_train, y_valid = prepare_data(
        x_train, x_valid, y_train, y_valid, neuronCats
    )
    return x_train, x_valid, y_train, y_valid


# --- Plots ---
def plot_loss(title, losses, scores):
    plt.rcParams["axes.grid"] = True
    fig, (ax1) = plt.subplots(1, 1, figsize=(8, 4))
    fig.suptitle("CVNN - Moons")
    ax1.plot(np.linspace(1, len(losses), len(losses)), losses)
    ax1.set_xlabel("Epoch")
    ax1.set_xlim(0, len(losses))

    ax1.plot(np.linspace(1, len(scores), len(scores)), scores)
    ax1.set_xlabel("Epoch")
    ax1.set_xlim(0, len(losses))

    ax1.legend(["Acc", "Loss"])

    plt.show()


def plot_weights(title, ylabel_1, ylabel_2, weights_real, weights_imag):
    # y_min = np.min([np.min(weights_real), np.min(weights_imag)])
    # y_max = np.max([np.max(weights_real), np.max(weights_imag)])

    fig, ax = plt.subplots(ncols=2, nrows=1, figsize=(14, 3))
    fig.suptitle(title)
    ax[0].plot(np.linspace(1, len(weights_real), len(weights_real)), weights_real)
    ax[0].set_xlabel("Step")
    ax[0].set_ylabel(ylabel_1)
    # ax[0].set_title("Real Valued Weigts")
    ax[0].set_xlim(0, len(weights_real))
    # ax[0].set_ylim(y_min, y_max)

    ax[1].plot(np.linspace(1, len(weights_imag), len(weights_imag)), weights_imag)
    ax[1].set_xlabel("Step")
    ax[1].set_ylabel(ylabel_2)
    # ax[1].set_title("Imaginary Valued Weights")
    ax[1].set_xlim(0, len(weights_imag))
    # ax[1].set_ylim(y_min, y_max)

    plt.show()


def plot_loss_acc_list(title, list_losses, list_scores, image_name):
    losses = np.mean(list_losses, axis=0)
    scores = np.mean(list_scores, axis=0)

    losses_std = np.std(list_losses, axis=0)
    scores_std = np.std(list_scores, axis=0)

    fig, (ax1) = plt.subplots(1, 1, figsize=(10, 3))
    fig.suptitle(title)
    ax1.plot(np.linspace(1, len(losses), len(losses)), losses)
    ax1.fill_between(
        np.linspace(1, len(losses), len(losses)),
        losses + losses_std,
        losses - losses_std,
        alpha=0.5,
        linewidth=0,
    )

    ax1.plot(np.linspace(1, len(scores), len(scores)), scores)
    ax1.fill_between(
        np.linspace(1, len(scores), len(scores)),
        scores + scores_std,
        scores - scores_std,
        alpha=0.5,
        linewidth=0,
    )
    ax1.set_xlabel("Epoch")

    plt.legend(["Loss Mean", "Loss Std", "Acc. Mean", "Acc. Std"])
    fig.savefig(image_name, format="png", dpi=600)

    plt.show()
    # save
    # fig.savefig(image_name + ".svg", format="svg", dpi=600)


# --- Logging ---
model_dict: dict = {}


def fc_hook(layer_name, module, grad_input, grad_output):
    if layer_name in model_dict:
        model_dict[layer_name]["weights"] = module.weights.detach().clone()
        model_dict[layer_name]["bias"] = module.bias.detach().clone()
        model_dict[layer_name]["grad_input"] = grad_input
        model_dict[layer_name]["grad_output"] = grad_output
    else:
        model_dict[layer_name] = {}
        model_dict[layer_name]["weights"] = module.weights.detach().clone()
        model_dict[layer_name]["bias"] = module.bias.detach().clone()
        model_dict[layer_name]["grad_input"] = grad_input
        model_dict[layer_name]["grad_output"] = grad_output

In [ ]:
# | hide
# control variables
# number of categories a neuron can distinguish / parameter that determines the number of output neurons
neuronCats = 1
# number of categories per neuron, i.e. neuronCats (+ 1 for others in case of multiple Outputs)
categories = 2
# how often a classification sector occurs (1 means no periodicity)
periodicity = 1
# path to store best model parameters

## Load Data

In [ ]:
train_csv = pd.read_csv(
    "data/autass_data2.csv",
    header=None,
    dtype=np.double,
)
data = np.array(train_csv.values[:, 1:50])
del train_csv

In [ ]:
X = data[:, 0:48]
y = data[:, 48].astype(int) - 1

yt = copy.copy(y)
yt[yt == 0] = 20
yt[yt == 1] = 21
yt[yt == 2] = 22
yt[yt == 3] = 23
yt[yt == 4] = 26
yt[yt == 5] = 24
yt[yt == 6] = 27
yt[yt == 7] = 29
yt[yt == 8] = 30
yt[yt == 9] = 25
yt[yt == 10] = 28
yt -= 20
y = yt
del yt

## Config

In [ ]:
epochs = 200
batch_size = 538
lr = 1
clip_angle_value = 1000000

## Single Layer

### MLMVN [48-10-11]

In [ ]:
PATH = str(Path.cwd() / "models/autass-mlmvn_48-10-11.pt")

In [ ]:
class Model(nn.Module):
    def __init__(self, categories, periodicity):
        super().__init__()
        self.categories = categories
        self.periodicity = periodicity
        self.first_linear = FirstLayer(48, 10)
        self.phase_act1 = cmplx_phase_activation()
        self.linear_out = OutputLayer(10, 11)
        self.phase_act2 = cmplx_phase_activation()
        # Hooks
        self.first_layer_hook_handle = self.first_linear.register_full_backward_hook(
            self.first_layer_backward_hook
        )
        self.output_hook_handle = self.linear_out.register_full_backward_hook(
            self.output_layer_backward_hook
        )

    def forward(self, x):
        x = self.first_linear(x)
        x = self.phase_act1(x)
        x = self.linear_out(x)
        x = self.phase_act2(x)
        return x

    def first_layer_backward_hook(self, module, grad_input, grad_output):
        fc_hook("first_layer", module, grad_input, grad_output)

    def hidden_layer_backward_hook(self, module, grad_input, grad_output):
        fc_hook("hidden_layer", module, grad_input, grad_output)

    def output_layer_backward_hook(self, module, grad_input, grad_output):
        fc_hook("output_layer", module, grad_input, grad_output)

    def angle2class(self, x: torch.tensor) -> torch.tensor:
        tmp = x.angle() + 2 * np.pi
        angle = torch.remainder(tmp, 2 * np.pi)

        # This will be the discrete output (the number of sector)
        o = torch.floor(self.categories * self.periodicity * angle / (2 * np.pi))
        return torch.remainder(o, self.categories)

    def predict(self, x):
        """
        Performs the prediction task of the network

        Args:
          x: torch.Tensor
            Input tensor of size ([3])

        Returns:
          Most likely class i.e., Label with the highest score
        """
        # Pass the data through the networks
        output = self.forward(x)

        # # Choose the label with the highest score
        # return torch.argmax(output, 1)
        return self.angle2class(output)


def fit(model, X, y, epochs, batch_size, optimizer, criterion, categories, periodicity):
    # List of losses for visualization
    losses = []
    scores = []
    acc_best = 0

    for i in range(epochs):
        # Pass the data through the network and compute the loss
        # We'll use the whole dataset during the training instead of using batches
        # in to order to keep the code simple for now.

        batch_loss = []

        for j in range((X.shape[0] - 1) // batch_size + 1):
            start_j = j * batch_size
            end_j = start_j + batch_size
            xb = X[start_j:end_j]
            yb = y[start_j:end_j]

            y_pred = model(xb)
            loss = criterion(y_pred, yb, categories, periodicity)
            batch_loss.append((torch.abs(loss)).detach().numpy())

            optimizer.zero_grad()
            loss.backward()
            optimizer.step(inputs=xb, layers=list(model.children()))

        losses.append(sum(batch_loss) / len(batch_loss))
        if i % 10 == 9:
            print(f"Epoch {i} loss is {losses[-1]}")
        y_pred = model.predict(X)
        scores.append(accuracy(y_pred.squeeze(), y))

        Logger.current_logger().report_scalar(
            "Loss/Acc", "Loss", iteration=i, value=losses[-1]
        )
        writer.add_scalar("Loss", losses[-1], i)
        Logger.current_logger().report_scalar(
            "Loss/Acc", "Acc", iteration=i, value=scores[-1]
        )
        writer.add_scalar("Accuracy", scores[-1], i)

        for key in model_dict:
            for key_layer in model_dict[key]:
                if key_layer in ["weights", "bias"]:
                    log_label = str(key) + "_" + str(key_layer)
                    log_label.replace(" ", "")
                    writer.add_histogram(
                        log_label + "_real", model_dict[key][key_layer].real, i
                    )
                    writer.add_histogram(
                        log_label + "_imag", model_dict[key][key_layer].imag, i
                    )
                    writer.add_histogram(
                        log_label + "_mag", torch.abs(model_dict[key][key_layer]), i
                    )
                    writer.add_histogram(
                        log_label + "_angle", torch.angle(model_dict[key][key_layer]), i
                    )

        if scores[-1] > acc_best:
            acc_best = scores[-1]
            torch.save(model.state_dict(), PATH)

    writer.close()
    return losses, scores

In [ ]:
model = Model(categories=categories, periodicity=periodicity)
criterion = ComplexMSE_adjusted_error.apply
optimizer = ECL(model.parameters(), lr=lr, clip_angle_value=clip_angle_value)

In [ ]:
task = Task.init(
    project_name="mlmvn",
    task_name="SDD-mlmvn-[48-10-11]",
    tags=["mlmvn", "SDD", "single_run", "adjusted_loss_clip_angle_value"],
)
writer = SummaryWriter()

#  capture a dictionary of hyperparameters with config
config_dict = {
    "learning_rate": 1,
    "epochs": epochs,
    "batch_size": batch_size,
    "optim": "ECL",
    "categories": categories,
    "periodicity": periodicity,
    "layer": "[48-10-11]",
    "loss": "ComplexMSE_adjusted_error",
    "clip_angle_value": clip_angle_value,
}
task.connect(config_dict)

ClearML Task: created new task id=605b9b0974d14aa08dac5c7e3519c776
ClearML results page: http://194.94.231.172:8080/projects/cdefd6ee85454e49be01962ad715eca0/experiments/605b9b0974d14aa08dac5c7e3519c776/output/log


{'learning_rate': 1,
 'epochs': 200,
 'batch_size': 538,
 'optim': 'ECL',
 'categories': 2,
 'periodicity': 1,
 'layer': '[48-10-11]',
 'loss': 'ComplexMSE_adjusted_error',
 'clip_angle_value': 1000000}

In [ ]:
x_train, x_valid, y_train, y_valid = get_splitted_data(X, y, neuronCats)

losses, scores = fit(
    model,
    x_train,
    y_train,
    epochs=epochs,
    batch_size=batch_size,
    optimizer=optimizer,
    criterion=criterion,
    categories=categories,
    periodicity=periodicity,
)

model.load_state_dict(torch.load(PATH))

y_pred = model.predict(x_train)
acc = accuracy(y_pred.squeeze(), y_train)
print("Train Acc.: ", acc)
Logger.current_logger().report_single_value(
    name="Train Acc.",
    value=acc,
)

y_pred = model.predict(x_valid)
acc = accuracy(y_pred.squeeze(), y_valid)
print("Val Acc.: ", acc)
Logger.current_logger().report_single_value(
    name="Val Acc.",
    value=acc,
)
print(classification_report(y_valid, y_pred.detach().numpy(), zero_division=0))

/tmp/ipykernel_4428/161459083.py:46: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



2022-09-22 22:34:24,956 - clearml.frameworks - INFO - Found existing registered model id=caa96da5a415490ca1ea0f95b383f403 [/home/antonpfeifer/Documents/mlmvn/nbs/examples/autass/models/autass-mlmvn_48-10-11.pt] reusing it.
Epoch 9 loss is 0.2580754619244136
Epoch 19 loss is 0.21683012693345186
Epoch 29 loss is 0.24423590104284812
Epoch 39 loss is 0.27051340395520046
Epoch 49 loss is 0.2290261834515428
Epoch 59 loss is 0.2985538988735455
Epoch 69 loss is 0.3595716436054556
Epoch 79 loss is 0.4444515348247725
Epoch 89 loss is 0.38659444896523715
Epoch 99 loss is 0.38398743291105353
Epoch 109 loss is 0.3754874847210009
Epoch 119 loss is 0.3750323965195017
Epoch 129 loss is 0.37294807279904635
Epoch 139 loss is 0.37278288673665216
Epoch 149 loss is 0.37282104403105726
Epoch 159 loss is 0.3728814936841066
Epoch 169 loss is 0.37325801241579976
Epoch 179 loss is 0.37343925531907135
Epoch 189 loss is 0.3743756613217295
Epoch 199 loss is 0.3748686972032385
Train Acc.:  0.8230782378327565
Val Ac

In [ ]:
task.mark_completed()
task.close()

### MLMVN [48-20-11]

In [ ]:
PATH = str(Path.cwd() / "models/autass-mlmvn_48-20-11.pt")

In [ ]:
class Model(nn.Module):
    def __init__(self, categories, periodicity):
        super().__init__()
        self.categories = categories
        self.periodicity = periodicity
        self.first_linear = FirstLayer(48, 20)
        self.phase_act1 = cmplx_phase_activation()
        self.linear_out = OutputLayer(20, 11)
        self.phase_act2 = cmplx_phase_activation()
        # Hooks
        self.first_layer_hook_handle = self.first_linear.register_full_backward_hook(
            self.first_layer_backward_hook
        )
        self.output_hook_handle = self.linear_out.register_full_backward_hook(
            self.output_layer_backward_hook
        )

    def forward(self, x):
        x = self.first_linear(x)
        x = self.phase_act1(x)
        x = self.linear_out(x)
        x = self.phase_act2(x)
        return x

    def first_layer_backward_hook(self, module, grad_input, grad_output):
        fc_hook("first_layer", module, grad_input, grad_output)

    def hidden_layer_backward_hook(self, module, grad_input, grad_output):
        fc_hook("hidden_layer", module, grad_input, grad_output)

    def output_layer_backward_hook(self, module, grad_input, grad_output):
        fc_hook("output_layer", module, grad_input, grad_output)

    def angle2class(self, x: torch.tensor) -> torch.tensor:
        tmp = x.angle() + 2 * np.pi
        angle = torch.remainder(tmp, 2 * np.pi)

        # This will be the discrete output (the number of sector)
        o = torch.floor(self.categories * self.periodicity * angle / (2 * np.pi))
        return torch.remainder(o, self.categories)

    def predict(self, x):
        """
        Performs the prediction task of the network

        Args:
          x: torch.Tensor
            Input tensor of size ([3])

        Returns:
          Most likely class i.e., Label with the highest score
        """
        # Pass the data through the networks
        output = self.forward(x)

        # # Choose the label with the highest score
        # return torch.argmax(output, 1)
        return self.angle2class(output)


def fit(model, X, y, epochs, batch_size, optimizer, criterion, categories, periodicity):
    # List of losses for visualization
    losses = []
    scores = []
    acc_best = 0

    for i in range(epochs):
        # Pass the data through the network and compute the loss
        # We'll use the whole dataset during the training instead of using batches
        # in to order to keep the code simple for now.

        batch_loss = []

        for j in range((X.shape[0] - 1) // batch_size + 1):
            start_j = j * batch_size
            end_j = start_j + batch_size
            xb = X[start_j:end_j]
            yb = y[start_j:end_j]

            y_pred = model(xb)
            loss = criterion(y_pred, yb, categories, periodicity)
            batch_loss.append((torch.abs(loss)).detach().numpy())

            optimizer.zero_grad()
            loss.backward()
            optimizer.step(inputs=xb, layers=list(model.children()))

        losses.append(sum(batch_loss) / len(batch_loss))
        if i % 10 == 9:
            print(f"Epoch {i} loss is {losses[-1]}")
        y_pred = model.predict(X)
        scores.append(accuracy(y_pred.squeeze(), y))

        Logger.current_logger().report_scalar(
            "Loss/Acc", "Loss", iteration=i, value=losses[-1]
        )
        writer.add_scalar("Loss", losses[-1], i)
        Logger.current_logger().report_scalar(
            "Loss/Acc", "Acc", iteration=i, value=scores[-1]
        )
        writer.add_scalar("Accuracy", scores[-1], i)

        for key in model_dict:
            for key_layer in model_dict[key]:
                if key_layer in ["weights", "bias"]:
                    log_label = str(key) + "_" + str(key_layer)
                    log_label.replace(" ", "")
                    writer.add_histogram(
                        log_label + "_real", model_dict[key][key_layer].real, i
                    )
                    writer.add_histogram(
                        log_label + "_imag", model_dict[key][key_layer].imag, i
                    )
                    writer.add_histogram(
                        log_label + "_mag", torch.abs(model_dict[key][key_layer]), i
                    )
                    writer.add_histogram(
                        log_label + "_angle", torch.angle(model_dict[key][key_layer]), i
                    )

        # writer.add_histogram("distribution centers", x + n_iter, i)
        if scores[-1] > acc_best:
            acc_best = scores[-1]
            torch.save(model.state_dict(), PATH)

    writer.close()
    return losses, scores

In [ ]:
model = Model(categories=categories, periodicity=periodicity)
criterion = ComplexMSE_adjusted_error.apply
optimizer = ECL(model.parameters(), lr=lr)

In [ ]:
task = Task.init(
    project_name="mlmvn",
    task_name="SDD-mlmvn-[48-20-11]",
    tags=["mlmvn", "SDD", "single_run", "adjusted_loss_clip_angle_value"],
)
writer = SummaryWriter()

#  capture a dictionary of hyperparameters with config
config_dict = {
    "learning_rate": 1,
    "epochs": epochs,
    "batch_size": batch_size,
    "optim": "ECL",
    "categories": categories,
    "periodicity": periodicity,
    "layer": "[48-20-11]",
    "loss": "ComplexMSE_adjusted_error",
    "clip_angle_value": clip_angle_value,
}
task.connect(config_dict)

ClearML Task: created new task id=24a6f11cfe9346f89c139a54af6465f8
ClearML results page: http://194.94.231.172:8080/projects/cdefd6ee85454e49be01962ad715eca0/experiments/24a6f11cfe9346f89c139a54af6465f8/output/log


{'learning_rate': 1,
 'epochs': 200,
 'batch_size': 538,
 'optim': 'ECL',
 'categories': 2,
 'periodicity': 1,
 'layer': '[48-20-11]',
 'loss': 'ComplexMSE_adjusted_error',
 'clip_angle_value': 1000000}

In [ ]:
x_train, x_valid, y_train, y_valid = get_splitted_data(X, y, neuronCats)

losses, scores = fit(
    model,
    x_train,
    y_train,
    epochs=epochs,
    batch_size=batch_size,
    optimizer=optimizer,
    criterion=criterion,
    categories=categories,
    periodicity=periodicity,
)

model.load_state_dict(torch.load(PATH))

y_pred = model.predict(x_train)
acc = accuracy(y_pred.squeeze(), y_train)
print("Train Acc.: ", acc)
Logger.current_logger().report_single_value(
    name="Train Acc.",
    value=acc,
)

y_pred = model.predict(x_valid)
acc = accuracy(y_pred.squeeze(), y_valid)
print("Val Acc.: ", acc)
Logger.current_logger().report_single_value(
    name="Val Acc.",
    value=acc,
)
print(classification_report(y_valid, y_pred.detach().numpy(), zero_division=0))

/tmp/ipykernel_4428/161459083.py:46: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



2022-09-22 22:37:34,712 - clearml.frameworks - INFO - Found existing registered model id=c337b94a22444d809d449783726d8ee2 [/home/antonpfeifer/Documents/mlmvn/nbs/examples/autass/models/autass-mlmvn_48-20-11.pt] reusing it.
Epoch 9 loss is 0.24476062344530095
Epoch 19 loss is 0.17524997121406452
Epoch 29 loss is 0.17326589625589803
Epoch 39 loss is 0.1250403376833135
Epoch 49 loss is 0.10869102016395354
Epoch 59 loss is 0.12357040169954497
Epoch 69 loss is 0.1093577461378208
Epoch 79 loss is 0.12643380921745745
Epoch 89 loss is 0.11328518219237213
Epoch 99 loss is 0.10813876603811017
Epoch 109 loss is 0.10728799886326053
Epoch 119 loss is 0.09921635073504151
Epoch 129 loss is 0.1086356377434932
Epoch 139 loss is 0.10239754285657386
Epoch 149 loss is 0.09024639101528273
Epoch 159 loss is 0.10786458972685774
Epoch 169 loss is 0.10565815323356467
Epoch 179 loss is 0.10199340886787205
Epoch 189 loss is 0.10471597717547888
Epoch 199 loss is 0.10269092835171191
Train Acc.:  0.9379994017860958

In [ ]:
task.mark_completed()
task.close()

### MLMVN [48-50-11]

In [ ]:
PATH = str(Path.cwd() / "models/autass-mlmvn_48-50-11.pt")

In [ ]:
class Model(nn.Module):
    def __init__(self, categories, periodicity):
        super().__init__()
        self.categories = categories
        self.periodicity = periodicity
        self.first_linear = FirstLayer(48, 50)
        self.phase_act1 = cmplx_phase_activation()
        self.linear_out = OutputLayer(50, 11)
        self.phase_act2 = cmplx_phase_activation()
        # Hooks
        self.first_layer_hook_handle = self.first_linear.register_full_backward_hook(
            self.first_layer_backward_hook
        )
        self.output_hook_handle = self.linear_out.register_full_backward_hook(
            self.output_layer_backward_hook
        )

    def forward(self, x):
        x = self.first_linear(x)
        x = self.phase_act1(x)
        x = self.linear_out(x)
        x = self.phase_act2(x)
        return x

    def first_layer_backward_hook(self, module, grad_input, grad_output):
        fc_hook("first_layer", module, grad_input, grad_output)

    def hidden_layer_backward_hook(self, module, grad_input, grad_output):
        fc_hook("hidden_layer", module, grad_input, grad_output)

    def output_layer_backward_hook(self, module, grad_input, grad_output):
        fc_hook("output_layer", module, grad_input, grad_output)

    def angle2class(self, x: torch.tensor) -> torch.tensor:
        tmp = x.angle() + 2 * np.pi
        angle = torch.remainder(tmp, 2 * np.pi)

        # This will be the discrete output (the number of sector)
        o = torch.floor(self.categories * self.periodicity * angle / (2 * np.pi))
        return torch.remainder(o, self.categories)

    def predict(self, x):
        """
        Performs the prediction task of the network

        Args:
          x: torch.Tensor
            Input tensor of size ([3])

        Returns:
          Most likely class i.e., Label with the highest score
        """
        # Pass the data through the networks
        output = self.forward(x)

        # # Choose the label with the highest score
        # return torch.argmax(output, 1)
        return self.angle2class(output)


def fit(model, X, y, epochs, batch_size, optimizer, criterion, categories, periodicity):
    # List of losses for visualization
    losses = []
    scores = []
    acc_best = 0

    for i in range(epochs):
        # Pass the data through the network and compute the loss
        # We'll use the whole dataset during the training instead of using batches
        # in to order to keep the code simple for now.

        batch_loss = []

        for j in range((X.shape[0] - 1) // batch_size + 1):
            start_j = j * batch_size
            end_j = start_j + batch_size
            xb = X[start_j:end_j]
            yb = y[start_j:end_j]

            y_pred = model(xb)
            loss = criterion(y_pred, yb, categories, periodicity)
            batch_loss.append((torch.abs(loss)).detach().numpy())

            optimizer.zero_grad()
            loss.backward()
            optimizer.step(inputs=xb, layers=list(model.children()))

        losses.append(sum(batch_loss) / len(batch_loss))
        if i % 10 == 9:
            print(f"Epoch {i} loss is {losses[-1]}")
        y_pred = model.predict(X)
        scores.append(accuracy(y_pred.squeeze(), y))

        Logger.current_logger().report_scalar(
            "Loss/Acc", "Loss", iteration=i, value=losses[-1]
        )
        writer.add_scalar("Loss", losses[-1], i)
        Logger.current_logger().report_scalar(
            "Loss/Acc", "Acc", iteration=i, value=scores[-1]
        )
        writer.add_scalar("Accuracy", scores[-1], i)

        for key in model_dict:
            for key_layer in model_dict[key]:
                if key_layer in ["weights", "bias"]:
                    log_label = str(key) + "_" + str(key_layer)
                    log_label.replace(" ", "")
                    writer.add_histogram(
                        log_label + "_real", model_dict[key][key_layer].real, i
                    )
                    writer.add_histogram(
                        log_label + "_imag", model_dict[key][key_layer].imag, i
                    )
                    writer.add_histogram(
                        log_label + "_mag", torch.abs(model_dict[key][key_layer]), i
                    )
                    writer.add_histogram(
                        log_label + "_angle", torch.angle(model_dict[key][key_layer]), i
                    )

        # writer.add_histogram("distribution centers", x + n_iter, i)
        if scores[-1] > acc_best:
            acc_best = scores[-1]
            torch.save(model.state_dict(), PATH)

    writer.close()
    return losses, scores

In [ ]:
model = Model(categories=categories, periodicity=periodicity)
criterion = ComplexMSE_adjusted_error.apply
optimizer = ECL(model.parameters(), lr=lr)

In [ ]:
task = Task.init(
    project_name="mlmvn",
    task_name="SDD-mlmvn-[48-50-11]",
    tags=["mlmvn", "SDD", "single_run", "adjusted_loss_clip_angle_value"],
)
writer = SummaryWriter()

#  capture a dictionary of hyperparameters with config
config_dict = {
    "learning_rate": 1,
    "epochs": epochs,
    "batch_size": batch_size,
    "optim": "ECL",
    "categories": categories,
    "periodicity": periodicity,
    "layer": "[48-50-11]",
    "loss": "ComplexMSE_adjusted_error",
    "clip_angle_value": clip_angle_value,
}
task.connect(config_dict)

ClearML Task: created new task id=6702fbaba3574fb49613b2911035194b
ClearML results page: http://194.94.231.172:8080/projects/cdefd6ee85454e49be01962ad715eca0/experiments/6702fbaba3574fb49613b2911035194b/output/log


{'learning_rate': 1,
 'epochs': 200,
 'batch_size': 538,
 'optim': 'ECL',
 'categories': 2,
 'periodicity': 1,
 'layer': '[48-50-11]',
 'loss': 'ComplexMSE_adjusted_error',
 'clip_angle_value': 1000000}

In [ ]:
x_train, x_valid, y_train, y_valid = get_splitted_data(X, y, neuronCats)

losses, scores = fit(
    model,
    x_train,
    y_train,
    epochs=epochs,
    batch_size=batch_size,
    optimizer=optimizer,
    criterion=criterion,
    categories=categories,
    periodicity=periodicity,
)

model.load_state_dict(torch.load(PATH))

y_pred = model.predict(x_train)
acc = accuracy(y_pred.squeeze(), y_train)
print("Train Acc.: ", acc)
Logger.current_logger().report_single_value(
    name="Train Acc.",
    value=acc,
)

y_pred = model.predict(x_valid)
acc = accuracy(y_pred.squeeze(), y_valid)
print("Val Acc.: ", acc)
Logger.current_logger().report_single_value(
    name="Val Acc.",
    value=acc,
)
print(classification_report(y_valid, y_pred.detach().numpy(), zero_division=0))

/tmp/ipykernel_4428/161459083.py:46: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



2022-09-22 22:41:10,743 - clearml.frameworks - INFO - Found existing registered model id=bb96e63090904339bf87c4852d30bdb6 [/home/antonpfeifer/Documents/mlmvn/nbs/examples/autass/models/autass-mlmvn_48-50-11.pt] reusing it.
Epoch 9 loss is 0.23981090673183392
Epoch 19 loss is 0.1350937123107344
Epoch 29 loss is 0.10171720961260702
Epoch 39 loss is 0.10397829965296723
Epoch 49 loss is 0.09251700067492151
Epoch 59 loss is 0.07677589171885311
Epoch 69 loss is 0.07428011077625409
Epoch 79 loss is 0.07424593369765878
Epoch 89 loss is 0.06974386731484813
Epoch 99 loss is 0.0625939765088816
Epoch 109 loss is 0.06981443338641498
Epoch 119 loss is 0.06748007620098263
Epoch 129 loss is 0.05657005163013897
Epoch 139 loss is 0.06359593600342729
Epoch 149 loss is 0.05271322280799125
Epoch 159 loss is 0.046950683837239236
Epoch 169 loss is 0.045860277729382115
Epoch 179 loss is 0.041940594701453865
Epoch 189 loss is 0.042831541937416555
Epoch 199 loss is 0.04799205469350261
Train Acc.:  0.97696876468

In [ ]:
task.mark_completed()
task.close()

### MLMVN [48-100-11]

In [ ]:
PATH = str(Path.cwd() / "models/autass-mlmvn_48-100-11.pt")

In [ ]:
class Model(nn.Module):
    def __init__(self, categories, periodicity):
        super().__init__()
        self.categories = categories
        self.periodicity = periodicity
        self.first_linear = FirstLayer(48, 100)
        self.phase_act1 = cmplx_phase_activation()
        self.linear_out = OutputLayer(100, 11)
        self.phase_act2 = cmplx_phase_activation()
        # Hooks
        self.first_layer_hook_handle = self.first_linear.register_full_backward_hook(
            self.first_layer_backward_hook
        )
        self.output_hook_handle = self.linear_out.register_full_backward_hook(
            self.output_layer_backward_hook
        )

    def forward(self, x):
        x = self.first_linear(x)
        x = self.phase_act1(x)
        x = self.linear_out(x)
        x = self.phase_act2(x)
        return x

    def first_layer_backward_hook(self, module, grad_input, grad_output):
        fc_hook("first_layer", module, grad_input, grad_output)

    def hidden_layer_backward_hook(self, module, grad_input, grad_output):
        fc_hook("hidden_layer", module, grad_input, grad_output)

    def output_layer_backward_hook(self, module, grad_input, grad_output):
        fc_hook("output_layer", module, grad_input, grad_output)

    def angle2class(self, x: torch.tensor) -> torch.tensor:
        tmp = x.angle() + 2 * np.pi
        angle = torch.remainder(tmp, 2 * np.pi)

        # This will be the discrete output (the number of sector)
        o = torch.floor(self.categories * self.periodicity * angle / (2 * np.pi))
        return torch.remainder(o, self.categories)

    def predict(self, x):
        """
        Performs the prediction task of the network

        Args:
          x: torch.Tensor
            Input tensor of size ([3])

        Returns:
          Most likely class i.e., Label with the highest score
        """
        # Pass the data through the networks
        output = self.forward(x)

        # # Choose the label with the highest score
        # return torch.argmax(output, 1)
        return self.angle2class(output)


def fit(model, X, y, epochs, batch_size, optimizer, criterion, categories, periodicity):
    # List of losses for visualization
    losses = []
    scores = []
    acc_best = 0

    for i in range(epochs):
        # Pass the data through the network and compute the loss
        # We'll use the whole dataset during the training instead of using batches
        # in to order to keep the code simple for now.

        batch_loss = []

        for j in range((X.shape[0] - 1) // batch_size + 1):
            start_j = j * batch_size
            end_j = start_j + batch_size
            xb = X[start_j:end_j]
            yb = y[start_j:end_j]

            y_pred = model(xb)
            loss = criterion(y_pred, yb, categories, periodicity)
            batch_loss.append((torch.abs(loss)).detach().numpy())

            optimizer.zero_grad()
            loss.backward()
            optimizer.step(inputs=xb, layers=list(model.children()))

        losses.append(sum(batch_loss) / len(batch_loss))
        if i % 10 == 9:
            print(f"Epoch {i} loss is {losses[-1]}")
        y_pred = model.predict(X)
        scores.append(accuracy(y_pred.squeeze(), y))

        Logger.current_logger().report_scalar(
            "Loss/Acc", "Loss", iteration=i, value=losses[-1]
        )
        writer.add_scalar("Loss", losses[-1], i)
        Logger.current_logger().report_scalar(
            "Loss/Acc", "Acc", iteration=i, value=scores[-1]
        )
        writer.add_scalar("Accuracy", scores[-1], i)

        for key in model_dict:
            for key_layer in model_dict[key]:
                if key_layer in ["weights", "bias"]:
                    log_label = str(key) + "_" + str(key_layer)
                    log_label.replace(" ", "")
                    writer.add_histogram(
                        log_label + "_real", model_dict[key][key_layer].real, i
                    )
                    writer.add_histogram(
                        log_label + "_imag", model_dict[key][key_layer].imag, i
                    )
                    writer.add_histogram(
                        log_label + "_mag", torch.abs(model_dict[key][key_layer]), i
                    )
                    writer.add_histogram(
                        log_label + "_angle", torch.angle(model_dict[key][key_layer]), i
                    )

        # writer.add_histogram("distribution centers", x + n_iter, i)
        if scores[-1] > acc_best:
            acc_best = scores[-1]
            torch.save(model.state_dict(), PATH)

    writer.close()
    return losses, scores

In [ ]:
model = Model(categories=categories, periodicity=periodicity)
criterion = ComplexMSE_adjusted_error.apply
optimizer = ECL(model.parameters(), lr=lr)

In [ ]:
task = Task.init(
    project_name="mlmvn",
    task_name="SDD-mlmvn-[48-100-11]",
    tags=["mlmvn", "SDD", "single_run", "adjusted_loss_clip_angle_value"],
)
writer = SummaryWriter()

#  capture a dictionary of hyperparameters with config
config_dict = {
    "learning_rate": 1,
    "epochs": epochs,
    "batch_size": batch_size,
    "optim": "ECL",
    "categories": categories,
    "periodicity": periodicity,
    "layer": "[48-100-11]",
    "loss": "ComplexMSE_adjusted_error",
    "clip_angle_value": clip_angle_value,
}
task.connect(config_dict)

ClearML Task: created new task id=ce81f8c4a4d444eb8de416771703c3f3
ClearML results page: http://194.94.231.172:8080/projects/cdefd6ee85454e49be01962ad715eca0/experiments/ce81f8c4a4d444eb8de416771703c3f3/output/log


{'learning_rate': 1,
 'epochs': 200,
 'batch_size': 538,
 'optim': 'ECL',
 'categories': 2,
 'periodicity': 1,
 'layer': '[48-100-11]',
 'loss': 'ComplexMSE_adjusted_error',
 'clip_angle_value': 1000000}

In [ ]:
x_train, x_valid, y_train, y_valid = get_splitted_data(X, y, neuronCats)

losses, scores = fit(
    model,
    x_train,
    y_train,
    epochs=epochs,
    batch_size=batch_size,
    optimizer=optimizer,
    criterion=criterion,
    categories=categories,
    periodicity=periodicity,
)

model.load_state_dict(torch.load(PATH))

y_pred = model.predict(x_train)
acc = accuracy(y_pred.squeeze(), y_train)
print("Train Acc.: ", acc)
Logger.current_logger().report_single_value(
    name="Train Acc.",
    value=acc,
)

y_pred = model.predict(x_valid)
acc = accuracy(y_pred.squeeze(), y_valid)
print("Val Acc.: ", acc)
Logger.current_logger().report_single_value(
    name="Val Acc.",
    value=acc,
)
print(classification_report(y_valid, y_pred.detach().numpy(), zero_division=0))

/tmp/ipykernel_4428/161459083.py:46: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



2022-09-22 22:45:48,058 - clearml.frameworks - INFO - Found existing registered model id=0f73e6db01fc42988672e4f44c0add5f [/home/antonpfeifer/Documents/mlmvn/nbs/examples/autass/models/autass-mlmvn_48-100-11.pt] reusing it.
Epoch 9 loss is 0.19675702136134435
Epoch 19 loss is 0.12691860293194868
Epoch 29 loss is 0.09963694001283371
Epoch 39 loss is 0.08266232549928246
Epoch 49 loss is 0.07721942842928552
Epoch 59 loss is 0.07040153803166502
Epoch 69 loss is 0.06772988802277076
Epoch 79 loss is 0.06808096965213609
Epoch 89 loss is 0.06277160699643843
Epoch 99 loss is 0.06304586082936255
Epoch 109 loss is 0.05947888365928943
Epoch 119 loss is 0.056828984384148414
Epoch 129 loss is 0.05502314843135909
Epoch 139 loss is 0.04954769492662839
Epoch 149 loss is 0.04978222914593748
Epoch 159 loss is 0.05478295147543594
Epoch 169 loss is 0.04662054887355213
Epoch 179 loss is 0.05929304252375296
Epoch 189 loss is 0.05981274953089023
Epoch 199 loss is 0.05613814120733131
Train Acc.:  0.97491774558

In [ ]:
task.mark_completed()
task.close()

## Multi Layer

### MLMVN [48-10-10-11]

In [ ]:
PATH = str(Path.cwd() / "models/autass-mlmvn_48-10-10-11.pt")

In [ ]:
class Model(nn.Module):
    def __init__(self, categories, periodicity):
        super().__init__()
        self.categories = categories
        self.periodicity = periodicity
        self.first_linear = FirstLayer(48, 10)
        self.phase_act1 = cmplx_phase_activation()
        self.hidden_layer = HiddenLayer(10, 10)
        self.phase_act2 = cmplx_phase_activation()
        self.linear_out = OutputLayer(10, 11)
        self.phase_act3 = cmplx_phase_activation()
        # Hooks
        self.first_layer_hook_handle = self.first_linear.register_full_backward_hook(
            self.first_layer_backward_hook
        )
        self.hidden_layer_hook_handle = self.hidden_layer.register_full_backward_hook(
            self.hidden_layer_backward_hook
        )
        self.output_hook_handle = self.linear_out.register_full_backward_hook(
            self.output_layer_backward_hook
        )

    def forward(self, x):
        x = self.first_linear(x)
        x = self.phase_act1(x)
        x = self.hidden_layer(x)
        x = self.phase_act2(x)
        x = self.linear_out(x)
        x = self.phase_act3(x)
        return x

    def first_layer_backward_hook(self, module, grad_input, grad_output):
        fc_hook("first_layer", module, grad_input, grad_output)

    def hidden_layer_backward_hook(self, module, grad_input, grad_output):
        fc_hook("hidden_layer", module, grad_input, grad_output)

    def output_layer_backward_hook(self, module, grad_input, grad_output):
        fc_hook("output_layer", module, grad_input, grad_output)

    def angle2class(self, x: torch.tensor) -> torch.tensor:
        tmp = x.angle() + 2 * np.pi
        angle = torch.remainder(tmp, 2 * np.pi)

        # This will be the discrete output (the number of sector)
        o = torch.floor(self.categories * self.periodicity * angle / (2 * np.pi))
        return torch.remainder(o, self.categories)

    def predict(self, x):
        """
        Performs the prediction task of the network

        Args:
          x: torch.Tensor
            Input tensor of size ([3])

        Returns:
          Most likely class i.e., Label with the highest score
        """
        # Pass the data through the networks
        output = self.forward(x)

        # # Choose the label with the highest score
        # return torch.argmax(output, 1)
        return self.angle2class(output)


def fit(model, X, y, epochs, batch_size, optimizer, criterion, categories, periodicity):
    # List of losses for visualization
    losses = []
    scores = []
    acc_best = 0

    for i in range(epochs):
        # Pass the data through the network and compute the loss
        # We'll use the whole dataset during the training instead of using batches
        # in to order to keep the code simple for now.

        batch_loss = []

        for j in range((X.shape[0] - 1) // batch_size + 1):
            start_j = j * batch_size
            end_j = start_j + batch_size
            xb = X[start_j:end_j]
            yb = y[start_j:end_j]

            y_pred = model(xb)
            loss = criterion(y_pred, yb, categories, periodicity)
            batch_loss.append((torch.abs(loss)).detach().numpy())

            optimizer.zero_grad()
            loss.backward()
            optimizer.step(inputs=xb, layers=list(model.children()))

        losses.append(sum(batch_loss) / len(batch_loss))
        if i % 10 == 9:
            print(f"Epoch {i} loss is {losses[-1]}")
        y_pred = model.predict(X)
        scores.append(accuracy(y_pred.squeeze(), y))

        Logger.current_logger().report_scalar(
            "Loss/Acc", "Loss", iteration=i, value=losses[-1]
        )
        writer.add_scalar("Loss", losses[-1], i)
        Logger.current_logger().report_scalar(
            "Loss/Acc", "Acc", iteration=i, value=scores[-1]
        )
        writer.add_scalar("Accuracy", scores[-1], i)

        for key in model_dict:
            for key_layer in model_dict[key]:
                if key_layer in ["weights", "bias"]:
                    log_label = str(key) + "_" + str(key_layer)
                    log_label.replace(" ", "")
                    writer.add_histogram(
                        log_label + "_real", model_dict[key][key_layer].real, i
                    )
                    writer.add_histogram(
                        log_label + "_imag", model_dict[key][key_layer].imag, i
                    )
                    writer.add_histogram(
                        log_label + "_mag", torch.abs(model_dict[key][key_layer]), i
                    )
                    writer.add_histogram(
                        log_label + "_angle", torch.angle(model_dict[key][key_layer]), i
                    )

        if scores[-1] > acc_best:
            acc_best = scores[-1]
            torch.save(model.state_dict(), PATH)

    writer.close()
    return losses, scores

In [ ]:
model = Model(categories=categories, periodicity=periodicity)
criterion = ComplexMSE_adjusted_error.apply
optimizer = ECL(model.parameters(), lr=lr)

In [ ]:
task = Task.init(
    project_name="mlmvn",
    task_name="SDD-mlmvn-[48-10-10-11]",
    tags=["mlmvn", "SDD", "single_run", "adjusted_loss_clip_angle_value"],
)
writer = SummaryWriter()

#  capture a dictionary of hyperparameters with config
config_dict = {
    "learning_rate": 1,
    "epochs": epochs,
    "batch_size": batch_size,
    "optim": "ECL",
    "categories": categories,
    "periodicity": periodicity,
    "layer": "[48-10-10-11]",
    "loss": "ComplexMSE_adjusted_error",
    "clip_angle_value": clip_angle_value,
}
task.connect(config_dict)

ClearML Task: created new task id=a6128e9bc7c64377a790bf906a1bb800
ClearML results page: http://194.94.231.172:8080/projects/cdefd6ee85454e49be01962ad715eca0/experiments/a6128e9bc7c64377a790bf906a1bb800/output/log


{'learning_rate': 1,
 'epochs': 200,
 'batch_size': 538,
 'optim': 'ECL',
 'categories': 2,
 'periodicity': 1,
 'layer': '[48-10-10-11]',
 'loss': 'ComplexMSE_adjusted_error',
 'clip_angle_value': 1000000}

In [ ]:
x_train, x_valid, y_train, y_valid = get_splitted_data(X, y, neuronCats)

losses, scores = fit(
    model,
    x_train,
    y_train,
    epochs=epochs,
    batch_size=batch_size,
    optimizer=optimizer,
    criterion=criterion,
    categories=categories,
    periodicity=periodicity,
)

model.load_state_dict(torch.load(PATH))

y_pred = model.predict(x_train)
acc = accuracy(y_pred.squeeze(), y_train)
print("Train Acc.: ", acc)
Logger.current_logger().report_single_value(
    name="Train Acc.",
    value=acc,
)

y_pred = model.predict(x_valid)
acc = accuracy(y_pred.squeeze(), y_valid)
print("Val Acc.: ", acc)
Logger.current_logger().report_single_value(
    name="Val Acc.",
    value=acc,
)
print(classification_report(y_valid, y_pred.detach().numpy(), zero_division=0))

/tmp/ipykernel_4428/161459083.py:46: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



2022-09-22 22:52:36,115 - clearml.frameworks - INFO - Found existing registered model id=410edb2915b24269b7d34f2e38593dff [/home/antonpfeifer/Documents/mlmvn/nbs/examples/autass/models/autass-mlmvn_48-10-10-11.pt] reusing it.
Epoch 9 loss is 0.5352860994082685
Epoch 19 loss is 0.8573220308220786
Epoch 29 loss is 0.8846553943619413
Epoch 39 loss is 0.9282479485173039
Epoch 49 loss is 0.9304585998948428
Epoch 59 loss is 0.8896554528724867
Epoch 69 loss is 0.9020712551731598
Epoch 79 loss is 0.8780626206812697
Epoch 89 loss is 0.9192603465644281
Epoch 99 loss is 0.9080221700448701
Epoch 109 loss is 0.8979155400653883
Epoch 119 loss is 0.9165361645534543
Epoch 129 loss is 0.8875104726098959
Epoch 139 loss is 0.9474128621314876
Epoch 149 loss is 0.8798036890266178
Epoch 159 loss is 0.870499564750111
Epoch 169 loss is 0.8812218883029092
Epoch 179 loss is 0.8765249923026144
Epoch 189 loss is 0.8565456302997135
Epoch 199 loss is 0.8486741464838758
Train Acc.:  0.5802033927274282
Val Acc.:  0.5

In [ ]:
task.mark_completed()
task.close()

### MLMVN [48-20-20-11]

In [ ]:
PATH = str(Path.cwd() / "models/autass-mlmvn_48-20-20-11.pt")

In [ ]:
class Model(nn.Module):
    def __init__(self, categories, periodicity):
        super().__init__()
        self.categories = categories
        self.periodicity = periodicity
        self.first_linear = FirstLayer(48, 20)
        self.phase_act1 = cmplx_phase_activation()
        self.hidden_layer = HiddenLayer(20, 20)
        self.phase_act2 = cmplx_phase_activation()
        self.linear_out = OutputLayer(20, 11)
        self.phase_act3 = cmplx_phase_activation()
        # Hooks
        self.first_layer_hook_handle = self.first_linear.register_full_backward_hook(
            self.first_layer_backward_hook
        )
        self.hidden_layer_hook_handle = self.hidden_layer.register_full_backward_hook(
            self.hidden_layer_backward_hook
        )
        self.output_hook_handle = self.linear_out.register_full_backward_hook(
            self.output_layer_backward_hook
        )

    def forward(self, x):
        x = self.first_linear(x)
        x = self.phase_act1(x)
        x = self.hidden_layer(x)
        x = self.phase_act2(x)
        x = self.linear_out(x)
        x = self.phase_act3(x)
        return x

    def first_layer_backward_hook(self, module, grad_input, grad_output):
        fc_hook("first_layer", module, grad_input, grad_output)

    def hidden_layer_backward_hook(self, module, grad_input, grad_output):
        fc_hook("hidden_layer", module, grad_input, grad_output)

    def output_layer_backward_hook(self, module, grad_input, grad_output):
        fc_hook("output_layer", module, grad_input, grad_output)

    def angle2class(self, x: torch.tensor) -> torch.tensor:
        tmp = x.angle() + 2 * np.pi
        angle = torch.remainder(tmp, 2 * np.pi)

        # This will be the discrete output (the number of sector)
        o = torch.floor(self.categories * self.periodicity * angle / (2 * np.pi))
        return torch.remainder(o, self.categories)

    def predict(self, x):
        """
        Performs the prediction task of the network

        Args:
          x: torch.Tensor
            Input tensor of size ([3])

        Returns:
          Most likely class i.e., Label with the highest score
        """
        # Pass the data through the networks
        output = self.forward(x)

        # # Choose the label with the highest score
        # return torch.argmax(output, 1)
        return self.angle2class(output)


def fit(model, X, y, epochs, batch_size, optimizer, criterion, categories, periodicity):
    # List of losses for visualization
    losses = []
    scores = []
    acc_best = 0

    for i in range(epochs):
        # Pass the data through the network and compute the loss
        # We'll use the whole dataset during the training instead of using batches
        # in to order to keep the code simple for now.

        batch_loss = []

        for j in range((X.shape[0] - 1) // batch_size + 1):
            start_j = j * batch_size
            end_j = start_j + batch_size
            xb = X[start_j:end_j]
            yb = y[start_j:end_j]

            y_pred = model(xb)
            loss = criterion(y_pred, yb, categories, periodicity)
            batch_loss.append((torch.abs(loss)).detach().numpy())

            optimizer.zero_grad()
            loss.backward()
            optimizer.step(inputs=xb, layers=list(model.children()))

        losses.append(sum(batch_loss) / len(batch_loss))
        if i % 10 == 9:
            print(f"Epoch {i} loss is {losses[-1]}")
        y_pred = model.predict(X)
        scores.append(accuracy(y_pred.squeeze(), y))

        Logger.current_logger().report_scalar(
            "Loss/Acc", "Loss", iteration=i, value=losses[-1]
        )
        writer.add_scalar("Loss", losses[-1], i)
        Logger.current_logger().report_scalar(
            "Loss/Acc", "Acc", iteration=i, value=scores[-1]
        )
        writer.add_scalar("Accuracy", scores[-1], i)

        for key in model_dict:
            for key_layer in model_dict[key]:
                if key_layer in ["weights", "bias"]:
                    log_label = str(key) + "_" + str(key_layer)
                    log_label.replace(" ", "")
                    writer.add_histogram(
                        log_label + "_real", model_dict[key][key_layer].real, i
                    )
                    writer.add_histogram(
                        log_label + "_imag", model_dict[key][key_layer].imag, i
                    )
                    writer.add_histogram(
                        log_label + "_mag", torch.abs(model_dict[key][key_layer]), i
                    )
                    writer.add_histogram(
                        log_label + "_angle", torch.angle(model_dict[key][key_layer]), i
                    )

        # writer.add_histogram("distribution centers", x + n_iter, i)
        if scores[-1] > acc_best:
            acc_best = scores[-1]
            torch.save(model.state_dict(), PATH)

    writer.close()
    return losses, scores

In [ ]:
model = Model(categories=categories, periodicity=periodicity)
criterion = ComplexMSE_adjusted_error.apply
optimizer = ECL(model.parameters(), lr=lr)

In [ ]:
task = Task.init(
    project_name="mlmvn",
    task_name="SDD-mlmvn-[48-20-20-11]",
    tags=["mlmvn", "SDD", "single_run", "adjusted_loss_clip_angle_value"],
)
writer = SummaryWriter()

#  capture a dictionary of hyperparameters with config
config_dict = {
    "learning_rate": 1,
    "epochs": epochs,
    "batch_size": batch_size,
    "optim": "ECL",
    "categories": categories,
    "periodicity": periodicity,
    "layer": "[48-20-20-11]",
    "loss": "ComplexMSE_adjusted_error",
    "clip_angle_value": clip_angle_value,
}
task.connect(config_dict)

ClearML Task: created new task id=b237a248793b43efbec3300c283c1b12
ClearML results page: http://194.94.231.172:8080/projects/cdefd6ee85454e49be01962ad715eca0/experiments/b237a248793b43efbec3300c283c1b12/output/log


{'learning_rate': 1,
 'epochs': 200,
 'batch_size': 538,
 'optim': 'ECL',
 'categories': 2,
 'periodicity': 1,
 'layer': '[48-20-20-11]',
 'loss': 'ComplexMSE_adjusted_error',
 'clip_angle_value': 1000000}

In [ ]:
x_train, x_valid, y_train, y_valid = get_splitted_data(X, y, neuronCats)

losses, scores = fit(
    model,
    x_train,
    y_train,
    epochs=epochs,
    batch_size=batch_size,
    optimizer=optimizer,
    criterion=criterion,
    categories=categories,
    periodicity=periodicity,
)

model.load_state_dict(torch.load(PATH))

y_pred = model.predict(x_train)
acc = accuracy(y_pred.squeeze(), y_train)
print("Train Acc.: ", acc)
Logger.current_logger().report_single_value(
    name="Train Acc.",
    value=acc,
)

y_pred = model.predict(x_valid)
acc = accuracy(y_pred.squeeze(), y_valid)
print("Val Acc.: ", acc)
Logger.current_logger().report_single_value(
    name="Val Acc.",
    value=acc,
)
print(classification_report(y_valid, y_pred.detach().numpy(), zero_division=0))

/tmp/ipykernel_4428/161459083.py:46: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



2022-09-22 22:56:11,994 - clearml.frameworks - INFO - Found existing registered model id=22ba5a4169ed406a9e74f40200bd29a1 [/home/antonpfeifer/Documents/mlmvn/nbs/examples/autass/models/autass-mlmvn_48-20-20-11.pt] reusing it.
Epoch 9 loss is 0.33310382354735135
Epoch 19 loss is 0.3045317527345633
Epoch 29 loss is 0.26208047114462196
Epoch 39 loss is 0.5633030280355086
Epoch 49 loss is 0.6604212666659914
Epoch 59 loss is 0.6350045622908638
Epoch 69 loss is 0.6371642678164225
Epoch 79 loss is 0.6625240454923111
Epoch 89 loss is 0.618989009951792
Epoch 99 loss is 0.5788566858980941
Epoch 109 loss is 0.5580994790124282
Epoch 119 loss is 0.5665000001440373
Epoch 129 loss is 0.6303235022304811
Epoch 139 loss is 0.6471938510996637
Epoch 149 loss is 0.6645542731394183
Epoch 159 loss is 0.6454662272330662
Epoch 169 loss is 0.6336318409892531
Epoch 179 loss is 0.6200715265569947
Epoch 189 loss is 0.6395423856912091
Epoch 199 loss is 0.5969744437662491
Train Acc.:  0.7452036063752511
Val Acc.:  0

In [ ]:
task.mark_completed()
task.close()

### MLMVN [48-50-50-11]

In [ ]:
PATH = str(Path.cwd() / "models/autass-mlmvn_48-50-50-11.pt")

In [ ]:
class Model(nn.Module):
    def __init__(self, categories, periodicity):
        super().__init__()
        self.categories = categories
        self.periodicity = periodicity
        self.first_linear = FirstLayer(48, 50)
        self.phase_act1 = cmplx_phase_activation()
        self.hidden_layer = HiddenLayer(50, 50)
        self.phase_act2 = cmplx_phase_activation()
        self.linear_out = OutputLayer(50, 11)
        self.phase_act3 = cmplx_phase_activation()
        # Hooks
        self.first_layer_hook_handle = self.first_linear.register_full_backward_hook(
            self.first_layer_backward_hook
        )
        self.hidden_layer_hook_handle = self.hidden_layer.register_full_backward_hook(
            self.hidden_layer_backward_hook
        )
        self.output_hook_handle = self.linear_out.register_full_backward_hook(
            self.output_layer_backward_hook
        )

    def forward(self, x):
        x = self.first_linear(x)
        x = self.phase_act1(x)
        x = self.hidden_layer(x)
        x = self.phase_act2(x)
        x = self.linear_out(x)
        x = self.phase_act3(x)
        return x

    def first_layer_backward_hook(self, module, grad_input, grad_output):
        fc_hook("first_layer", module, grad_input, grad_output)

    def hidden_layer_backward_hook(self, module, grad_input, grad_output):
        fc_hook("hidden_layer", module, grad_input, grad_output)

    def output_layer_backward_hook(self, module, grad_input, grad_output):
        fc_hook("output_layer", module, grad_input, grad_output)

    def angle2class(self, x: torch.tensor) -> torch.tensor:
        tmp = x.angle() + 2 * np.pi
        angle = torch.remainder(tmp, 2 * np.pi)

        # This will be the discrete output (the number of sector)
        o = torch.floor(self.categories * self.periodicity * angle / (2 * np.pi))
        return torch.remainder(o, self.categories)

    def predict(self, x):
        """
        Performs the prediction task of the network

        Args:
          x: torch.Tensor
            Input tensor of size ([3])

        Returns:
          Most likely class i.e., Label with the highest score
        """
        # Pass the data through the networks
        output = self.forward(x)

        # # Choose the label with the highest score
        # return torch.argmax(output, 1)
        return self.angle2class(output)


def fit(model, X, y, epochs, batch_size, optimizer, criterion, categories, periodicity):
    # List of losses for visualization
    losses = []
    scores = []
    acc_best = 0

    for i in range(epochs):
        # Pass the data through the network and compute the loss
        # We'll use the whole dataset during the training instead of using batches
        # in to order to keep the code simple for now.

        batch_loss = []

        for j in range((X.shape[0] - 1) // batch_size + 1):
            start_j = j * batch_size
            end_j = start_j + batch_size
            xb = X[start_j:end_j]
            yb = y[start_j:end_j]

            y_pred = model(xb)
            loss = criterion(y_pred, yb, categories, periodicity)
            batch_loss.append((torch.abs(loss)).detach().numpy())

            optimizer.zero_grad()
            loss.backward()
            optimizer.step(inputs=xb, layers=list(model.children()))

        losses.append(sum(batch_loss) / len(batch_loss))
        if i % 10 == 9:
            print(f"Epoch {i} loss is {losses[-1]}")
        y_pred = model.predict(X)
        scores.append(accuracy(y_pred.squeeze(), y))

        Logger.current_logger().report_scalar(
            "Loss/Acc", "Loss", iteration=i, value=losses[-1]
        )
        writer.add_scalar("Loss", losses[-1], i)
        Logger.current_logger().report_scalar(
            "Loss/Acc", "Acc", iteration=i, value=scores[-1]
        )
        writer.add_scalar("Accuracy", scores[-1], i)

        for key in model_dict:
            for key_layer in model_dict[key]:
                if key_layer in ["weights", "bias"]:
                    log_label = str(key) + "_" + str(key_layer)
                    log_label.replace(" ", "")
                    writer.add_histogram(
                        log_label + "_real", model_dict[key][key_layer].real, i
                    )
                    writer.add_histogram(
                        log_label + "_imag", model_dict[key][key_layer].imag, i
                    )
                    writer.add_histogram(
                        log_label + "_mag", torch.abs(model_dict[key][key_layer]), i
                    )
                    writer.add_histogram(
                        log_label + "_angle", torch.angle(model_dict[key][key_layer]), i
                    )

        # writer.add_histogram("distribution centers", x + n_iter, i)
        if scores[-1] > acc_best:
            acc_best = scores[-1]
            torch.save(model.state_dict(), PATH)

    writer.close()
    return losses, scores

In [ ]:
model = Model(categories=categories, periodicity=periodicity)
criterion = ComplexMSE_adjusted_error.apply
optimizer = ECL(model.parameters(), lr=lr)

In [ ]:
task = Task.init(
    project_name="mlmvn",
    task_name="SDD-mlmvn-[48-50-50-11]",
    tags=["mlmvn", "SDD", "single_run", "adjusted_loss_clip_angle_value"],
)
writer = SummaryWriter()

#  capture a dictionary of hyperparameters with config
config_dict = {
    "learning_rate": 1,
    "epochs": epochs,
    "batch_size": batch_size,
    "optim": "ECL",
    "categories": categories,
    "periodicity": periodicity,
    "layer": "[48-50-50-11]",
    "loss": "ComplexMSE_adjusted_error",
    "clip_angle_value": clip_angle_value,
}
task.connect(config_dict)

ClearML Task: created new task id=fd3418e73a054cb2882dd2956cc509d3
ClearML results page: http://194.94.231.172:8080/projects/cdefd6ee85454e49be01962ad715eca0/experiments/fd3418e73a054cb2882dd2956cc509d3/output/log


{'learning_rate': 1,
 'epochs': 200,
 'batch_size': 538,
 'optim': 'ECL',
 'categories': 2,
 'periodicity': 1,
 'layer': '[48-50-50-11]',
 'loss': 'ComplexMSE_adjusted_error',
 'clip_angle_value': 1000000}

In [ ]:
x_train, x_valid, y_train, y_valid = get_splitted_data(X, y, neuronCats)

losses, scores = fit(
    model,
    x_train,
    y_train,
    epochs=epochs,
    batch_size=batch_size,
    optimizer=optimizer,
    criterion=criterion,
    categories=categories,
    periodicity=periodicity,
)

model.load_state_dict(torch.load(PATH))

y_pred = model.predict(x_train)
acc = accuracy(y_pred.squeeze(), y_train)
print("Train Acc.: ", acc)
Logger.current_logger().report_single_value(
    name="Train Acc.",
    value=acc,
)

y_pred = model.predict(x_valid)
acc = accuracy(y_pred.squeeze(), y_valid)
print("Val Acc.: ", acc)
Logger.current_logger().report_single_value(
    name="Val Acc.",
    value=acc,
)
print(classification_report(y_valid, y_pred.detach().numpy(), zero_division=0))

/tmp/ipykernel_4428/161459083.py:46: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



2022-09-22 23:00:39,760 - clearml.frameworks - INFO - Found existing registered model id=f13061c5d03a4e96b788becd5e54443a [/home/antonpfeifer/Documents/mlmvn/nbs/examples/autass/models/autass-mlmvn_48-50-50-11.pt] reusing it.
Epoch 9 loss is 0.228445869053326
Epoch 19 loss is 0.1769025202901553
Epoch 29 loss is 0.13638551487260145
Epoch 39 loss is 0.12389236050933781
Epoch 49 loss is 0.10762237564656339
Epoch 59 loss is 0.09565055571155039
Epoch 69 loss is 0.08563549226626824
Epoch 79 loss is 0.07961991527154336
Epoch 89 loss is 0.08409453002137111
Epoch 99 loss is 0.11271795819004764
Epoch 109 loss is 0.09878662956475265
Epoch 119 loss is 0.0966812858882719
Epoch 129 loss is 0.08962629585333381
Epoch 139 loss is 0.08695483521719193
Epoch 149 loss is 0.08539091657127361
Epoch 159 loss is 0.09094220320688623
Epoch 169 loss is 0.09666773124329389
Epoch 179 loss is 0.0897644800428595
Epoch 189 loss is 0.09589186592403481
Epoch 199 loss is 0.09725851786233339
Train Acc.:  0.934452847925479

In [ ]:
task.mark_completed()
task.close()

### MLMVN [48-100-100-11]

In [ ]:
PATH = str(Path.cwd() / "models/autass-mlmvn_48-100-100-11.pt")

In [ ]:
class Model(nn.Module):
    def __init__(self, categories, periodicity):
        super().__init__()
        self.categories = categories
        self.periodicity = periodicity
        self.first_linear = FirstLayer(48, 100)
        self.phase_act1 = cmplx_phase_activation()
        self.hidden_layer = HiddenLayer(100, 100)
        self.phase_act2 = cmplx_phase_activation()
        self.linear_out = OutputLayer(100, 11)
        self.phase_act3 = cmplx_phase_activation()
        # Hooks
        self.first_layer_hook_handle = self.first_linear.register_full_backward_hook(
            self.first_layer_backward_hook
        )
        self.hidden_layer_hook_handle = self.hidden_layer.register_full_backward_hook(
            self.hidden_layer_backward_hook
        )
        self.output_hook_handle = self.linear_out.register_full_backward_hook(
            self.output_layer_backward_hook
        )

    def forward(self, x):
        x = self.first_linear(x)
        x = self.phase_act1(x)
        x = self.hidden_layer(x)
        x = self.phase_act2(x)
        x = self.linear_out(x)
        x = self.phase_act3(x)
        return x

    def first_layer_backward_hook(self, module, grad_input, grad_output):
        fc_hook("first_layer", module, grad_input, grad_output)

    def hidden_layer_backward_hook(self, module, grad_input, grad_output):
        fc_hook("hidden_layer", module, grad_input, grad_output)

    def output_layer_backward_hook(self, module, grad_input, grad_output):
        fc_hook("output_layer", module, grad_input, grad_output)

    def angle2class(self, x: torch.tensor) -> torch.tensor:
        tmp = x.angle() + 2 * np.pi
        angle = torch.remainder(tmp, 2 * np.pi)

        # This will be the discrete output (the number of sector)
        o = torch.floor(self.categories * self.periodicity * angle / (2 * np.pi))
        return torch.remainder(o, self.categories)

    def predict(self, x):
        """
        Performs the prediction task of the network

        Args:
          x: torch.Tensor
            Input tensor of size ([3])

        Returns:
          Most likely class i.e., Label with the highest score
        """
        # Pass the data through the networks
        output = self.forward(x)

        # # Choose the label with the highest score
        # return torch.argmax(output, 1)
        return self.angle2class(output)


def fit(model, X, y, epochs, batch_size, optimizer, criterion, categories, periodicity):
    # List of losses for visualization
    losses = []
    scores = []
    acc_best = 0

    for i in range(epochs):
        # Pass the data through the network and compute the loss
        # We'll use the whole dataset during the training instead of using batches
        # in to order to keep the code simple for now.

        batch_loss = []

        for j in range((X.shape[0] - 1) // batch_size + 1):
            start_j = j * batch_size
            end_j = start_j + batch_size
            xb = X[start_j:end_j]
            yb = y[start_j:end_j]

            y_pred = model(xb)
            loss = criterion(y_pred, yb, categories, periodicity)
            batch_loss.append((torch.abs(loss)).detach().numpy())

            optimizer.zero_grad()
            loss.backward()
            optimizer.step(inputs=xb, layers=list(model.children()))

        losses.append(sum(batch_loss) / len(batch_loss))
        if i % 10 == 9:
            print(f"Epoch {i} loss is {losses[-1]}")
        y_pred = model.predict(X)
        scores.append(accuracy(y_pred.squeeze(), y))

        Logger.current_logger().report_scalar(
            "Loss/Acc", "Loss", iteration=i, value=losses[-1]
        )
        writer.add_scalar("Loss", losses[-1], i)
        Logger.current_logger().report_scalar(
            "Loss/Acc", "Acc", iteration=i, value=scores[-1]
        )
        writer.add_scalar("Accuracy", scores[-1], i)

        for key in model_dict:
            for key_layer in model_dict[key]:
                if key_layer in ["weights", "bias"]:
                    log_label = str(key) + "_" + str(key_layer)
                    log_label.replace(" ", "")
                    writer.add_histogram(
                        log_label + "_real", model_dict[key][key_layer].real, i
                    )
                    writer.add_histogram(
                        log_label + "_imag", model_dict[key][key_layer].imag, i
                    )
                    writer.add_histogram(
                        log_label + "_mag", torch.abs(model_dict[key][key_layer]), i
                    )
                    writer.add_histogram(
                        log_label + "_angle", torch.angle(model_dict[key][key_layer]), i
                    )

        # writer.add_histogram("distribution centers", x + n_iter, i)
        if scores[-1] > acc_best:
            acc_best = scores[-1]
            torch.save(model.state_dict(), PATH)

    writer.close()
    return losses, scores

In [ ]:
model = Model(categories=categories, periodicity=periodicity)
criterion = ComplexMSE_adjusted_error.apply
optimizer = ECL(model.parameters(), lr=lr)

In [ ]:
task = Task.init(
    project_name="mlmvn",
    task_name="SDD-mlmvn-[48-100-100-11]",
    tags=["mlmvn", "SDD", "single_run", "adjusted_loss_clip_angle_value"],
)
writer = SummaryWriter()

#  capture a dictionary of hyperparameters with config
config_dict = {
    "learning_rate": lr,
    "epochs": epochs,
    "batch_size": batch_size,
    "optim": "ECL",
    "categories": categories,
    "periodicity": periodicity,
    "layer": "[48-100-100-11]",
    "loss": "ComplexMSE_adjusted_error",
    "clip_angle_value": clip_angle_value,
}
task.connect(config_dict)

ClearML Task: created new task id=8b2cde0afc44486290f4e94181fbdeed
ClearML results page: http://194.94.231.172:8080/projects/cdefd6ee85454e49be01962ad715eca0/experiments/8b2cde0afc44486290f4e94181fbdeed/output/log


{'learning_rate': 1,
 'epochs': 200,
 'batch_size': 538,
 'optim': 'ECL',
 'categories': 2,
 'periodicity': 1,
 'layer': '[48-100-100-11]',
 'loss': 'ComplexMSE_adjusted_error',
 'clip_angle_value': 1000000}

In [ ]:
x_train, x_valid, y_train, y_valid = get_splitted_data(X, y, neuronCats)

losses, scores = fit(
    model,
    x_train,
    y_train,
    epochs=epochs,
    batch_size=batch_size,
    optimizer=optimizer,
    criterion=criterion,
    categories=categories,
    periodicity=periodicity,
)

model.load_state_dict(torch.load(PATH))

y_pred = model.predict(x_train)
acc = accuracy(y_pred.squeeze(), y_train)
print("Train Acc.: ", acc)
Logger.current_logger().report_single_value(
    name="Train Acc.",
    value=acc,
)

y_pred = model.predict(x_valid)
acc = accuracy(y_pred.squeeze(), y_valid)
print("Val Acc.: ", acc)
Logger.current_logger().report_single_value(
    name="Val Acc.",
    value=acc,
)
print(classification_report(y_valid, y_pred.detach().numpy(), zero_division=0))

/tmp/ipykernel_4428/161459083.py:46: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



2022-09-22 23:07:15,673 - clearml.frameworks - INFO - Found existing registered model id=bbd65d869dea4025af46d264d3c7bdee [/home/antonpfeifer/Documents/mlmvn/nbs/examples/autass/models/autass-mlmvn_48-100-100-11.pt] reusing it.
Epoch 9 loss is 0.3850133302732534
Epoch 19 loss is 0.22928350502576889
Epoch 29 loss is 0.18473120489159367
Epoch 39 loss is 0.151906876717256
Epoch 49 loss is 0.13306868079012138
Epoch 59 loss is 0.12176301270741918
Epoch 69 loss is 0.11425308696387655
Epoch 79 loss is 0.1074521977072848
Epoch 89 loss is 0.09860157716737963
Epoch 99 loss is 0.09525496381491966
Epoch 109 loss is 0.09652682279150973
Epoch 119 loss is 0.09278257367639477
Epoch 129 loss is 0.09085598133404435
Epoch 139 loss is 0.0890178305214046
Epoch 149 loss is 0.08561603038010733
Epoch 159 loss is 0.08194544436834562
Epoch 169 loss is 0.07805339709847767
Epoch 179 loss is 0.08145146698926171
Epoch 189 loss is 0.07395142641579694
Epoch 199 loss is 0.07222291978897587
Train Acc.:  0.9455838995000

In [ ]:
task.mark_completed()
task.close()